In [1]:
import numpy as np
from entsoe import EntsoePandasClient
import pandas as pd
from functools import reduce

### API initialization

In [2]:
f = open('api_token', 'r')
token = f.read()

In [3]:
client = EntsoePandasClient(api_key=token)

### Use API to scrap data about solar and wind generation

In [248]:
def entsoe_api_generation(start_time: str, end_time: str, country: str, plant: str):
    start = pd.Timestamp(start_time, tz='Europe/Brussels')
    end = pd.Timestamp(end_time, tz='Europe/Brussels')
    h_in_year = (end - start).days * 24
    try:
        df = client.query_generation(country, start=start,end=end, psr_type=plant)
    except:
        df = pd.DataFrame(0, index=np.arange(8760), columns=[plant])
    if len(df) > h_in_year:
        i = 0
        stop = len(df) - 1
        new_agg = []
        while i<stop:
            var = (df.iloc[i,0] + df.iloc[i+1,0] + df.iloc[i+2,0] + df.iloc[i+3,0])/4
            new_agg.append(var)
            i = i + 4
        df = pd.DataFrame({plant:new_agg})
    else:
        df = df.iloc[:,0]
        df.reset_index(drop=True)
    return df

In [249]:
def entsoe_api_dataframe(start_time: str, end_time: str, country: str):
    plant_type = ["B16", "B18", "B19"]
    plant_idx = ["i10", "i8", "i3"]
    df_solar = entsoe_api_generation(start_time, end_time, country, plant_type[0]).reset_index(drop=True)
    df_offshore = entsoe_api_generation(start_time, end_time, country, plant_type[1]).reset_index(drop=True)
    df_onshore = entsoe_api_generation(start_time, end_time, country, plant_type[2]).reset_index(drop=True)
    data_frames = [df_solar, df_offshore, df_onshore]
    df_merged = reduce(lambda  left,right: pd.merge(left,right,left_index=True, right_index=True,
                                            how='outer'), data_frames)
    df_merged.columns = plant_idx
    return df_merged

In [250]:
def all_country(start_time = '20210101', end_time = '20220101'):
    df_de = entsoe_api_dataframe(start_time, end_time, "DE_LU").add_suffix('_de')
    df_france = entsoe_api_dataframe(start_time, end_time, "FR").add_suffix('_fr')
    df_be = entsoe_api_dataframe(start_time, end_time, "BE").reset_index(drop=True)
    df_nl = entsoe_api_dataframe(start_time, end_time, "NL")
    df_benelux = df_be + df_nl
    df_benelux = df_benelux.add_suffix("_bnl")
    df = df_de.join(df_france).join(df_benelux)
    df = df.fillna(0)
    return df

In [251]:
df = all_country()

<ipython-input-249-dea9633093bf>:8: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  df_merged = reduce(lambda  left,right: pd.merge(left,right,left_index=True, right_index=True,


In [263]:
df.head()

,i10_de,i8_de,i3_de,i10_fr,i8_fr,i3_fr,i10_bnl,i8_bnl,i3_bnl
0,0.0,383.00,3934.5,0.0,0,1082.0,0.0,474.00,166.75
1,0.0,394.50,3541.5,0.0,0,1075.0,0.0,549.50,174.25
2,0.0,305.25,3217.0,0.0,0,1086.0,0.0,558.50,190.75
3,0.0,319.25,2787.5,0.0,0,1149.0,0.0,580.75,223.50
4,0.0,296.25,2492.5,0.0,0,1170.0,0.0,615.75,256.75


In [257]:
df.to_excel("output.xlsx", sheet_name="Generation")

### Scrap load data

In [267]:
start = pd.Timestamp('20171201', tz='Europe/Brussels')
end = pd.Timestamp('20180101', tz='Europe/Brussels')
client.query_load("LU", start=start,end=end)

,Actual Load
2017-12-01 00:00:00+01:00,560.0
2017-12-01 00:15:00+01:00,546.0
2017-12-01 00:30:00+01:00,533.0
2017-12-01 00:45:00+01:00,525.0
2017-12-01 01:00:00+01:00,514.0
...,...
2017-12-31 22:45:00+01:00,418.0
2017-12-31 23:00:00+01:00,413.0
2017-12-31 23:15:00+01:00,463.0
2017-12-31 23:30:00+01:00,483.0


In [4]:
def entsoe_api_load(start_time: str, end_time: str, country: str):
    start = pd.Timestamp(start_time, tz='Europe/Brussels')
    end = pd.Timestamp(end_time, tz='Europe/Brussels')
    h_in_year = (end - start).days * 24
    try:
        df = client.query_load(country, start=start,end=end)
    except:
        df = pd.DataFrame(0, index=np.arange(8760), columns=[country])
    if len(df) > h_in_year:
        i = 0
        stop = len(df) - 1
        new_agg = []
        while i<stop:
            var = (df.iloc[i,0] + df.iloc[i+1,0] + df.iloc[i+2,0] + df.iloc[i+3,0])/4
            new_agg.append(var)
            i = i + 4
        df = pd.DataFrame({"Load":new_agg})
    else:
        df.columns = ["Load"]
        df.reset_index(drop=True)
    return df

In [24]:
def load_country(start_time = '20210101', end_time = '20220101'):
    df_de = entsoe_api_load(start_time, end_time, "DE").add_suffix('_de')
    df_france = entsoe_api_load(start_time, end_time, "FR").add_suffix('_fr').reset_index(drop=True)
    df_be = entsoe_api_load(start_time, end_time, "BE")
    df_nl = entsoe_api_load(start_time, end_time, "NL")
    df_lu = entsoe_api_load(start_time, end_time, "LU")
    df_benelux = df_be + df_nl + df_lu
    df_benelux = df_benelux.add_suffix("_bnl")
    df = df_de.join(df_france, how="outer").join(df_benelux)
    df = df.fillna(0)
    return df

In [25]:
df_load = load_country()

In [26]:
df_load

,Load_de,Load_fr,Load_bnl
0,44586.00,65761.0,21102.50
1,42832.25,64139.0,20441.00
2,41111.00,62657.0,19541.25
3,40364.50,59481.0,18834.00
4,40318.00,57656.0,18477.75
...,...,...,...
8755,55785.75,0.0,22331.25
8756,51849.00,0.0,21203.00
8757,48752.00,0.0,20126.00
8758,47461.50,0.0,0.00


In [39]:
start_time = '20210101'
end_time = '20220101'
df_france = entsoe_api_load(start_time, end_time, "FR")

In [40]:
df_france= df_france.reset_index(drop=True)

In [41]:
df_france

,Load
0,65761.0
1,64139.0
2,62657.0
3,59481.0
4,57656.0
...,...
8746,56693.0
8747,54155.0
8748,51945.0
8749,52907.0


In [42]:
df_load

,Load_de,Load_fr,Load_bnl
0,44586.00,65761.0,21102.50
1,42832.25,64139.0,20441.00
2,41111.00,62657.0,19541.25
3,40364.50,59481.0,18834.00
4,40318.00,57656.0,18477.75
...,...,...,...
8755,55785.75,0.0,22331.25
8756,51849.00,0.0,21203.00
8757,48752.00,0.0,20126.00
8758,47461.50,0.0,0.00


In [47]:
df_load['Load_bnl']=df_load['Load_bnl'].replace(0,df_load['Load_bnl'].median())

In [48]:
df_load

,Load_de,Load_fr,Load_bnl
0,44586.00,65761.0,21102.50
1,42832.25,64139.0,20441.00
2,41111.00,62657.0,19541.25
3,40364.50,59481.0,18834.00
4,40318.00,57656.0,18477.75
...,...,...,...
8755,55785.75,51121.0,22331.25
8756,51849.00,51121.0,21203.00
8757,48752.00,51121.0,20126.00
8758,47461.50,51121.0,21892.00


In [49]:
df_load.to_excel("load_output.xlsx", sheet_name="Load")